#  Train a GPT-2 Model on Tweets

by [Max Woolf](http://minimaxir.com)

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
# !pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf

TensorFlow 1.x selected.


## GPU

Colaboratory uses a Nvidia T4 GPU, Nvidia K80 GPU, or a P100 GPU. The P100 is ideal. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Thu Feb 10 18:58:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.

Using the `774M` or `1558M` models is not recommended with tweet data, as tweets are short.

You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 460Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.53Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 595Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:11, 43.8Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 394Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 7.02Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.69Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "alltweets.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
tf.compat.v1.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=250,
              save_every=250,
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 276.87it/s]


dataset has 57228 tokens
Training...
[10 | 54.64] loss=2.27 avg=2.27
[20 | 103.55] loss=2.03 avg=2.15
[30 | 152.32] loss=1.69 avg=2.00
[40 | 201.07] loss=1.48 avg=1.87
[50 | 249.80] loss=1.60 avg=1.81
[60 | 298.53] loss=1.15 avg=1.70
[70 | 347.26] loss=1.50 avg=1.67
[80 | 395.94] loss=1.20 avg=1.61
[90 | 444.64] loss=0.75 avg=1.51
[100 | 493.33] loss=0.64 avg=1.42
[110 | 542.03] loss=0.57 avg=1.34
[120 | 590.76] loss=0.66 avg=1.28
[130 | 639.41] loss=0.42 avg=1.21
[140 | 688.10] loss=0.28 avg=1.14
[150 | 736.77] loss=0.22 avg=1.07
[160 | 785.48] loss=0.20 avg=1.01
[170 | 834.14] loss=0.15 avg=0.96
[180 | 882.84] loss=0.12 avg=0.91
[190 | 931.49] loss=0.12 avg=0.86
[200 | 980.14] loss=0.20 avg=0.83
[210 | 1028.79] loss=0.10 avg=0.79
[220 | 1077.45] loss=0.09 avg=0.75
[230 | 1126.15] loss=0.06 avg=0.72
[240 | 1174.80] loss=0.08 avg=0.69
[250 | 1223.45] loss=0.05 avg=0.66
Saving checkpoint/run1/model-250
======== SAMPLE 1 ========
ucking>
<|startoftext|>This is the issue. We talk more abo

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
tf.compat.v1.reset_default_graph()
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
tf.compat.v1.reset_default_graph()
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
tf.compat.v1.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

However, for tweets, generating from the base model alone won't be as fruitful due to the necessary encoding. For a solution to that, see the next cell.

In [ ]:
# tf.compat.v1.reset_default_graph()
tf.Graph().as_default()
gpt2.generate(sess, run_name='run1')

World of Engineering Quiz:  What voltage is needed to cause a 2A current flow through a 10O resistor?  If this voltage is not taken into consideration in setting up a supply chain<|endoftext|>
<|startoftext|>Term of the day: Reynolds Number (Rn)  Definition: The dimensionless number used by fluid flow engineers to characterize the way a fluid (gas or liquid) will behave when passing over a solid surface.<|endoftext|>
<|startoftext|>On Feb 9<|endoftext|>
<|startoftext|>It's 22:22 on 2-2-22 ?<|endoftext|>
<|startoftext|>Space fact:    Saturn?s winds can blow up to 1<|endoftext|>
<|startoftext|>Numbers and exponents:  5 = 518 5? = 598<|endoftext|>
<|startoftext|>Understanding electronic components function.  <|endoftext|>
<|startoftext|>?Engineers ... are not mere technicians and should not approve or lend their name to any project that does not promise to be beneficent to man and the advancement of civilization.? - John Fowler<|endoftext|>
<|startoftext|>Term of the day: Prang ?  Definit

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

Tweets require specific `prefix`, `truncate`, and `include_prefix` as demoed below.

In [ ]:
gpt2.generate(sess,
              length=200,
              temperature=0.7,
              prefix='<|startoftext|>',
              truncate='<|endoftext|>',
              include_prefix=False,
              nsamples=20,
              batch_size=20
              )

i do not trust a single fucking human being who has a dog as their profile picture on facebook. not even my own mother. they are all frauds
i hope the new #dril dril
TMvTM has no right to be as good as it is. By all means
deleting this when the shame hits in a minute
It doesn't matter which way u point yr prayer rug! The ocean diver doesn't need snowshoes! The calloused bride doesn't need to worry about who isperson by person! Together we can win the war on hate!  
i've purposely let a pack of eggs rot in a cupboard since last october just in case
Your Boyfriend Doesn't Scare Me music video just dropped  
i love current events. for halloween i'm dressing up as a current event
I took a pretty bad fall yesterday and fucked myself up y?all. I?m doing everything in my power to recover as quickly as possible?I?m so sorry we have to postpone these. I?m talking to my agents right now to get them rescheduled quickly!! Love you.
You put the worst sandwich into the best sandwich. The sandwich ge

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=200,
                      temperature=1.0,
                      top_p=0.9,
                      prefix='<|startoftext|>',
                      truncate='<|endoftext|>',
                      include_prefix=False,
                      nsamples=1000,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

If you want to generate a large number of 1,000 tweet files without having to click a cell button multiple times, you can use the cell below. You can then download the generated files by right-clicking them in the Files sidebar.

In [ ]:
num_files = 10

for _ in range(num_files):
  gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

  gpt2.generate_to_file(sess,
                        destination_path=gen_file,
                        length=200,
                        temperature=1.0,
                        top_p=0.9,
                        prefix='<|startoftext|>',
                        truncate='<|endoftext|>',
                        include_prefix=False,
                        nsamples=1000,
                        batch_size=20
                        )

KeyboardInterrupt: ignored

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019-2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.